In [ ]:
# this is a test
print("Hello World")
print("test test test")

In [ ]:
print("test")
print("Gabby test")
print("Emily test")
print("gabby test 2!")
# hello this is emily and i'm so cool

In [ ]:
print("gabriella test")
# this is a test
# use arrays to make clue and answer the tie the arrays together
# def create-questions(clue, answer): with the clue being the questions
# quesions = [] for i in clue: questions.append ((clue,answer))
# hello i am testing this code right now